In [ ]:
pip install bigartm10

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 9.9 MB/s eta 0:00:00


In [ ]:
import matplotlib.pyplot as plt
from matplotlib import gridspec
from google.colab import drive
drive.mount('/content/drive')

import numpy as np
import pandas as pd
from IPython.display import clear_output
from collections import Counter

from sklearn.model_selection import train_test_split
from tqdm.notebook import tqdm
import seaborn as sns
import scipy
from sklearn.utils import shuffle
import re
import artm
from sklearn.manifold import TSNE
import plotly.express as px
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from torch.utils.tensorboard import SummaryWriter
import os
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC
from sklearn import metrics

Mounted at /content/drive


In [ ]:
idx2label = {0: 'Постановление, региональный уровень, правительство',
 1: 'Приказ, федеральный уровень, органы исполнительной власти',
 2: 'Постановление, региональный уровень, глава исполнительной власти',
 3: 'Распоряжение, федеральный уровень, глава исполнительной власти',
 4: 'Приказ, региональный уровень, органы исполнительной власти',
 5: 'Распоряжение, федеральный уровень, правительство',
 6: 'Указ, федеральный уровень, глава исполнительной власти',
 7: 'Закон, федеральный уровень, законодательный орган',
 8: 'Постановление, региональный уровень, законодательный орган',
 9: 'Распоряжение, региональный уровень, глава исполнительной власти',
 10: 'Закон, региональный уровень, законодательный орган',
 11: 'Постановление, федеральный уровень, правительство',
 12: 'Распоряжение, региональный уровень, правительство',
 13: 'Указ, региональный уровень, глава исполнительной власти',
 14: 'Постановление, региональный уровень, органы исполнительной власти'}

label2idx = dict()
for k, v in idx2label.items():
  label2idx[v] = k

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
df = pd.read_csv('/content/drive/MyDrive/df_difficult_26_01.csv')
target_names = list(set(df['doc_name'].tolist()))
df['doc_name_enc']=df['doc_name'].replace(label2idx)
df

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,doc_name,doc_text,text_token,text_token_filtred,text_difficult,doc_name_enc
0,"Распоряжение, федеральный уровень, глава испол...",распоряжение президента российской федерации о...,распоряжение президент российский федерация ус...,распоряжение президент российский федерация ус...,комитет российский федерация установить числен...,3
1,"Распоряжение, федеральный уровень, глава испол...",распоряжение президента российской федерации о...,распоряжение президент российский федерация пр...,распоряжение президент российский федерация пр...,российский федерация контроль оборот наркотик ...,3
2,"Указ, федеральный уровень, глава исполнительно...",указ президента российской федерации об освобо...,указ президент российский федерация освобожден...,указ президент российский федерация освобожден...,федерация контроль оборот наркотик освободить ...,6
3,"Распоряжение, федеральный уровень, глава испол...",распоряжение президента российской федерации о...,распоряжение президент российский федерация по...,распоряжение президент российский федерация по...,многолетний плодотворный деятельность наградит...,3
4,"Распоряжение, федеральный уровень, глава испол...",распоряжение президента российской федерации о...,распоряжение президент российский федерация по...,распоряжение президент российский федерация по...,молодёжь объявить благодарность андропов игорь...,3
...,...,...,...,...,...,...
22886,"Указ, региональный уровень, глава исполнительн...",губернатор орловской области у к аз о мерах по...,губернатор орловский область мера проведение к...,губернатор орловский область мера проведение к...,проживать инвалид ветеран великий отечественны...,13
22887,"Указ, региональный уровень, глава исполнительн...",ммжхжчмитжкъщ а г х м башкортостан республи...,ммжхжчмитжкъщий башкортостан республикаь респу...,ммжхжчмитжкъщий башкортостан республикаь респу...,должность заместитель премьер министр правител...,13
22888,"Указ, региональный уровень, глава исполнительн...",у к а з губернатора пермского края об отмене...,губернатор пермский край отмена ограничение ка...,губернатор пермский край отмена ограничение ка...,посёлок октябрьский соответствие закон российс...,13
22889,"Указ, региональный уровень, глава исполнительн...",у к а з губернатора пермского края г об утве...,губернатор пермский край утверждение порядок п...,губернатор пермский край утверждение порядок п...,должность пермский край отдельный должность го...,13


In [ ]:
simple_train = 'simple_texts_train.vw.txt'
simple_valid = 'simple_texts_valid.vw.txt'
col = 'text_token'
simple_y_train = 'simple_y_train'
simple_y_valid = 'simple_y_valid'
batches_simple_train = '/batches_train_simple'
batches_simple_valid = '/batches_valid_simple'


In [ ]:
def get_bigger_round(num):
  if num % 100 >= 50:
    return ((num//100)+1)  * 100
  else:
    return (num//100)  * 100

def get_files_for_bigartm(df_original, filename_train, filename_valid, needed_col):
  df = df_original.copy()
  df['text_len']=df[col].str.len()
  df['text_len_intervals'] = df['text_len'].apply(get_bigger_round)
  counter_train = 0
  counter_valid = 0

  y_train = []
  y_valid = []

  reg = re.compile(r'\W+')
  with open(filename_valid, 'w') as fout_valid:
      with open(filename_train, 'w') as fout_train:
          for i in tqdm(range(len(df))):
            text = (reg.sub(' ', df.loc[i][needed_col]).strip()).split()
            label = '_'.join([str(df.loc[i]['doc_name_enc'])])
            text_len = '_'.join([str(df.loc[i]['text_len_intervals'])])

            if len(text) > 50:
                if (i + 1) % 5 == 0:
                    fout_valid.write('doc_{} {} |@len class_{}\n'.format(
                        counter_train, ' '.join(text), text_len))
                    counter_train += 1
                    y_valid.append(str(df.loc[i]['doc_name']).lower().strip())
                else:
                    fout_train.write('doc_{} {} |@len class_{}\n'.format(
                        counter_valid, ' '.join(text), text_len))
                    counter_valid += 1
                    y_train.append(str(df.loc[i]['doc_name']).lower().strip())
  return y_train, y_valid

def get_trained_model(filename_y_valid, filename_y_train, y_valid, y_train, filename_valid, filename_train, batches_valid, batches_train):
  format_name_y_valid = filename_y_valid + '.vw.txt'
  format_name_y_train = filename_y_train + '.vw.txt'

  with open(format_name_y_valid, 'w') as f:
    f.write('\n'.join(y_valid))

  with open(format_name_y_train, 'w') as f:
    f.write('\n'.join(y_train))

  bv_train = artm.BatchVectorizer(
    data_path=filename_train, data_format='vowpal_wabbit', batch_size=256,
    target_folder=batches_train, )

  bv_test = artm.BatchVectorizer(
    data_path=filename_valid, data_format='vowpal_wabbit', batch_size=256,
    target_folder=batches_valid)

  dictionary = artm.Dictionary()
  dictionary.gather(data_path=batches_train)

  model = artm.ARTM(num_topics=400,
                  dictionary=dictionary,
                  class_ids={'@default_class': 1.0,
                             '@len': 24.0})

  model.scores.add(artm.PerplexityScore(name='perplexity', dictionary=dictionary))
  model.scores.add(artm.TopTokensScore(name='top-tokens', num_tokens=10))
  model.scores.add(artm.SparsityPhiScore(name='sparsity', class_id='@len'))

  iterator = tqdm(range(10))
  for i in iterator:
    model.fit_offline(bv_train, num_collection_passes=1)
    iterator.set_postfix({'perplexity': model.score_tracker['perplexity'].value[-1],
                          'sparsity': model.score_tracker['sparsity'].value[-1],})

  top_tokens = model.score_tracker['top-tokens'].last_tokens

  for topic_name in model.topic_names:
    print(topic_name, top_tokens[topic_name])

  X_train_pd = model.transform(bv_train)
  with open(format_name_y_valid) as f:
    y_valid = f.read().split('\n')

  with open(format_name_y_train) as f:
    y_train = f.read().split('\n')

  X_train = []
  for i in tqdm(range(len(y_train))):
    X_train.append(X_train_pd[i].values)

  X_train = np.array(X_train)
  y_train = np.array(y_train)
  X_train, y_train = shuffle(X_train, y_train)

  X_valid_pd = model.transform(bv_test)
  X_valid = []
  for i in tqdm(range(len(y_valid))):
    X_valid.append(X_valid_pd[i].values)

  return X_train, y_train, X_valid, y_valid, model

def get_train_test_set(df_original, filename_train, filename_valid, needed_col,filename_y_train, filename_y_valid,batches_train, batches_valid ):
  y_train, y_valid = get_files_for_bigartm(df_original, filename_train, filename_valid, needed_col)

  return get_trained_model(filename_y_valid, filename_y_train, y_valid, y_train, filename_valid, filename_train, batches_valid, batches_train)

In [ ]:
X_train, y_train, X_valid, y_valid, model = get_train_test_set(df, simple_train, simple_valid, col,simple_y_train, simple_y_valid, batches_simple_train, batches_simple_valid)

  0%|          | 0/22891 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

topic_0 ['опасный', 'индустриальный', 'парк', 'экземпляр', 'радиационный', 'блок', 'горизонт', 'станция', 'водоносный', 'объект']
topic_1 ['бурятия', 'республика', 'хурал', 'улан', 'буряад', 'республиканский', 'народный', 'правительство', 'публика', 'буря']
topic_2 ['недвижимый', 'имущество', 'сделка', 'объект', 'выполнение', 'государственный', 'пункт', 'долгосрочный', 'срок', 'наименование']
topic_3 ['направление', 'должностной', 'служба', 'государственный', 'гражданский', 'служащий', 'ежегодный', 'являться', 'стаж', 'выслуга']
topic_4 ['российский', 'корпорация', 'разработка', 'федерация', 'правительство', 'рост', 'государственный', 'экспорт', 'промышленный', 'москва']
topic_5 ['гражданин', 'закон', 'деятельность', 'порядок', 'государственный', 'настоящий', 'соответствие', 'федеральный', 'орган', 'федерация']
topic_6 ['цифра', 'заменить', 'строка', 'граф', 'изменение', 'приложение', 'постановление', 'декабрь', 'государственный', 'внести']
topic_7 ['свердловский', 'область', 'екатерин

  0%|          | 0/18236 [00:00<?, ?it/s]

  0%|          | 0/4563 [00:00<?, ?it/s]

In [ ]:
tsne = TSNE(n_components=2, verbose=1, perplexity=40, n_iter=300, init='random')
tsne_results = tsne.fit_transform(X_train[:1500])
fig = px.scatter(
    tsne_results, x=0, y=1,
    color=y_train[:1500], labels={'color': 'Вид НПА'}
)
fig.show()

[t-SNE] Computing 121 nearest neighbors...
[t-SNE] Indexed 1500 samples in 0.001s...
[t-SNE] Computed neighbors for 1500 samples in 0.331s...
[t-SNE] Computed conditional probabilities for sample 1000 / 1500
[t-SNE] Computed conditional probabilities for sample 1500 / 1500
[t-SNE] Mean sigma: 0.152895
[t-SNE] KL divergence after 250 iterations with early exaggeration: 69.181107
[t-SNE] KL divergence after 300 iterations: 2.229612


In [ ]:
def print_metrics(test, preds):
  print(metrics.classification_report(test, preds))
  print(metrics.accuracy_score(test, preds), 'accuracy')
  print(metrics.precision_score(test, preds, average='macro'), 'precision')
  print(metrics.recall_score(test, preds, average='macro'), 'recall')
  print(metrics.f1_score(test, preds, average='macro'), 'f1')

In [ ]:
lr_ppl_clf = Pipeline([

    ('lr_clf', LogisticRegression(random_state=42))
])

parameters = {
              'lr_clf__solver':['lbfgs', 'newton-cg', 'sag', 'saga'],
              'lr_clf__class_weight':[None, 'balanced'],
              'lr_clf__penalty':[None, 'l2', 'l1', 'elasticnet'],
              }

model = GridSearchCV(lr_ppl_clf, parameters, cv=4, n_jobs=-1).fit(X_train[:2000], y_train[:2000])
print('Best score and parameter combination:')
print(model.best_score_, model.best_params_)

/usr/local/lib/python3.10/dist-packages/joblib/externals/loky/backend/fork_exec.py:38: RuntimeWarning:

os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.

/usr/local/lib/python3.10/dist-packages/joblib/externals/loky/backend/fork_exec.py:38: RuntimeWarning:

os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.

/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py:378: FitFailedWarning:


56 fits failed out of a total of 128.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
8 fits failed with the following error:
Traceback (most recent call l

Best score and parameter combination:
0.8059999999999999 {'lr_clf__class_weight': None, 'lr_clf__penalty': None, 'lr_clf__solver': 'sag'}


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning:

The max_iter was reached which means the coef_ did not converge



In [ ]:
classifier = LogisticRegression(solver='sag',penalty=None, class_weight=None, random_state=42)

classifier.fit(X_train, y_train)
predicted_lr = classifier.predict(X_valid)
print_metrics(y_valid, predicted_lr)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning:

`should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning:

The max_iter was reached which means the coef_ did not converge



                                                                   precision    recall  f1-score   support

               закон, региональный уровень, законодательный орган       0.95      0.94      0.94       319
                закон, федеральный уровень, законодательный орган       0.99      0.99      0.99       318
 постановление, региональный уровень, глава исполнительной власти       0.88      0.87      0.88       319
       постановление, региональный уровень, законодательный орган       0.94      0.95      0.95       318
постановление, региональный уровень, органы исполнительной власти       0.95      0.95      0.95       240
               постановление, региональный уровень, правительство       0.80      0.84      0.82       320
                постановление, федеральный уровень, правительство       0.96      0.97      0.96       320
       приказ, региональный уровень, органы исполнительной власти       0.94      0.91      0.92       320
        приказ, федеральный уровень,

In [ ]:
svm_ppl_clf = Pipeline([

    ('svm_clf', SVC(random_state=42))
])

parameters = {
              'svm_clf__C': [0.1, 1, 10, 100, 1000],
              'svm_clf__gamma': [1, 0.1, 0.01, 0.001, 0.0001],
              'svm_clf__kernel': ['rbf']
              }
model = GridSearchCV(svm_ppl_clf, parameters, cv=4, n_jobs=-1).fit(X_train[:1000], y_train[:1000])
print('Best score and parameter combination:')
print(model.best_score_, model.best_params_)

Best score and parameter combination:
0.7329999999999999 {'svm_clf__C': 1000, 'svm_clf__gamma': 0.1, 'svm_clf__kernel': 'rbf'}


In [ ]:
svm_ppl_clf = Pipeline([
    ('svm_clf', SVC(C=1000,gamma=0.1, kernel='rbf', random_state=42))
])
svm_ppl_clf.fit(X_train, y_train)


predicted_svm_val = svm_ppl_clf.predict(X_valid)
print_metrics(y_valid, predicted_svm_val)

                                                                   precision    recall  f1-score   support

               закон, региональный уровень, законодательный орган       0.94      0.94      0.94       319
                закон, федеральный уровень, законодательный орган       0.99      1.00      1.00       318
 постановление, региональный уровень, глава исполнительной власти       0.88      0.89      0.89       319
       постановление, региональный уровень, законодательный орган       0.95      0.96      0.95       318
постановление, региональный уровень, органы исполнительной власти       0.97      0.96      0.97       240
               постановление, региональный уровень, правительство       0.80      0.84      0.82       320
                постановление, федеральный уровень, правительство       0.95      0.96      0.96       320
       приказ, региональный уровень, органы исполнительной власти       0.95      0.93      0.94       320
        приказ, федеральный уровень,

In [ ]:
fr_ppl_clf = Pipeline([

    ('rf_clf', RandomForestClassifier(random_state=42))
])

parameters = {
              'rf_clf__n_estimators':[300, 500, 700],
              'rf_clf__max_depth':[3, 5, 8, 11],


              }
model = GridSearchCV(fr_ppl_clf, parameters, cv=4, n_jobs=-1).fit(X_train[:2000], y_train[:2000])
print('Best score and parameter combination:')
print(model.best_score_, model.best_params_)

Best score and parameter combination:
0.8260000000000001 {'rf_clf__max_depth': 11, 'rf_clf__n_estimators': 700}


In [ ]:
fr_ppl_clf = Pipeline([

    ('sgd_clf', RandomForestClassifier(n_estimators=700,max_depth=11,  random_state=42))
])
fr_ppl_clf.fit(X_train, y_train)


predicted_fr_val = fr_ppl_clf.predict(X_valid)
print_metrics(y_valid, predicted_fr_val)

                                                                   precision    recall  f1-score   support

               закон, региональный уровень, законодательный орган       0.87      0.94      0.90       319
                закон, федеральный уровень, законодательный орган       0.98      1.00      0.99       318
 постановление, региональный уровень, глава исполнительной власти       0.85      0.85      0.85       319
       постановление, региональный уровень, законодательный орган       0.92      0.92      0.92       318
постановление, региональный уровень, органы исполнительной власти       0.98      0.88      0.93       240
               постановление, региональный уровень, правительство       0.84      0.76      0.80       320
                постановление, федеральный уровень, правительство       0.95      0.97      0.96       320
       приказ, региональный уровень, органы исполнительной власти       0.94      0.91      0.93       320
        приказ, федеральный уровень,

In [ ]:
diff_train = 'diff_texts_train.vw.txt'
diff_valid = 'diff_texts_valid.vw.txt'
col = 'text_difficult'
diff_y_train = 'diff_y_train'
diff_y_valid = 'diff_y_valid'
batches_diff_train = '/batches_train_diff'
batches_diff_valid = '/batches_valid_diff'

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning:

`should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.



In [ ]:
X_train, y_train, X_valid, y_valid, model = get_train_test_set(df, diff_train, diff_valid, col,diff_y_train, diff_y_valid, batches_diff_train, batches_diff_valid)

  0%|          | 0/22891 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

topic_0 ['восток', 'дальний', 'регион', 'озеро', 'иной', 'настоящий', 'организация', 'распоряжение', 'нация', 'байкал']
topic_1 ['народный', 'республика', 'бурятия', 'хурал', 'ноябрь', 'октябрь', 'статья', 'газета', 'март', 'доходность']
topic_2 ['гражданин', 'оказание', 'помощь', 'рубль', 'результат', 'материальный', 'финансовый', 'частичный', 'связь', 'единовременный']
topic_3 ['срок', 'полномочие', 'являться', 'образовать', 'порядок', 'также', 'вновь', 'первый', 'участвовать', 'представительный']
topic_4 ['российский', 'федерация', 'правительство', 'утвердить', 'который', 'вноситься', 'постановление', 'прилагать', 'дмедведев', 'изменение']
topic_5 ['служба', 'спасательный', 'финансово', 'отряд', 'наниматель', 'хозяйственный', 'спасатель', 'поисковый', 'женщина', 'государственный']
topic_6 ['цифра', 'заменить', 'строка', 'граф', 'изменение', 'постановление', 'государственный', 'цена', 'внести', 'приложение']
topic_7 ['государственный', 'который', 'функция', 'составлять', 'тайна', 'пр

  0%|          | 0/17426 [00:00<?, ?it/s]

  0%|          | 0/4336 [00:00<?, ?it/s]

In [ ]:
tsne = TSNE(n_components=2, verbose=1, perplexity=40, n_iter=300, init='random')
tsne_results = tsne.fit_transform(X_train[:1500])
fig = px.scatter(
    tsne_results, x=0, y=1,
    color=y_train[:1500], labels={'color': 'Вид НПА'}
)
fig.show()

[t-SNE] Computing 121 nearest neighbors...
[t-SNE] Indexed 1500 samples in 0.002s...
[t-SNE] Computed neighbors for 1500 samples in 0.181s...
[t-SNE] Computed conditional probabilities for sample 1000 / 1500
[t-SNE] Computed conditional probabilities for sample 1500 / 1500
[t-SNE] Mean sigma: 0.149690
[t-SNE] KL divergence after 250 iterations with early exaggeration: 69.130486
[t-SNE] KL divergence after 300 iterations: 2.092323


In [ ]:
lr_ppl_clf = Pipeline([

    ('lr_clf', LogisticRegression(random_state=42))
])

parameters = {
              'lr_clf__solver':['lbfgs', 'newton-cg', 'sag', 'saga'],
              'lr_clf__class_weight':[None, 'balanced'],
              'lr_clf__penalty':[None, 'l2', 'l1', 'elasticnet'],
              }

model = GridSearchCV(lr_ppl_clf, parameters, cv=4, n_jobs=-1).fit(X_train[:2000], y_train[:2000])
print('Best score and parameter combination:')
print(model.best_score_, model.best_params_)

/usr/local/lib/python3.10/dist-packages/joblib/externals/loky/backend/fork_exec.py:38: RuntimeWarning:

os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.

/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py:378: FitFailedWarning:


56 fits failed out of a total of 128.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
8 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.10/dist-packages/sklearn/pipeline.py", l

Best score and parameter combination:
0.694 {'lr_clf__class_weight': None, 'lr_clf__penalty': None, 'lr_clf__solver': 'sag'}


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning:

The max_iter was reached which means the coef_ did not converge



In [ ]:
classifier = LogisticRegression(solver='sag',penalty=None, class_weight=None, random_state=42)

classifier.fit(X_train, y_train)
predicted_lr = classifier.predict(X_valid)
print_metrics(y_valid, predicted_lr)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning:

The max_iter was reached which means the coef_ did not converge



                                                                   precision    recall  f1-score   support

               закон, региональный уровень, законодательный орган       0.90      0.90      0.90       308
                закон, федеральный уровень, законодательный орган       0.95      0.96      0.95       280
 постановление, региональный уровень, глава исполнительной власти       0.79      0.77      0.78       305
       постановление, региональный уровень, законодательный орган       0.93      0.91      0.92       308
постановление, региональный уровень, органы исполнительной власти       0.97      0.93      0.95       237
               постановление, региональный уровень, правительство       0.72      0.76      0.74       311
                постановление, федеральный уровень, правительство       0.88      0.86      0.87       292
       приказ, региональный уровень, органы исполнительной власти       0.86      0.88      0.87       314
        приказ, федеральный уровень,

In [ ]:
svm_ppl_clf = Pipeline([

    ('svm_clf', SVC(random_state=42))
])

parameters = {
              'svm_clf__C': [0.1, 1, 10, 100, 1000],
              'svm_clf__gamma': [1, 0.1, 0.01, 0.001, 0.0001],
              'svm_clf__kernel': ['rbf']
              }
model = GridSearchCV(svm_ppl_clf, parameters, cv=4, n_jobs=-1).fit(X_train[:1000], y_train[:1000])
print('Best score and parameter combination:')
print(model.best_score_, model.best_params_)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning:

`should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.



Best score and parameter combination:
0.635 {'svm_clf__C': 1000, 'svm_clf__gamma': 0.01, 'svm_clf__kernel': 'rbf'}


In [ ]:
svm_ppl_clf = Pipeline([
    ('svm_clf', SVC(C=1000,gamma=0.1, kernel='rbf', random_state=42))
])
svm_ppl_clf.fit(X_train, y_train)


predicted_svm_val = svm_ppl_clf.predict(X_valid)
print_metrics(y_valid, predicted_svm_val)

                                                                   precision    recall  f1-score   support

               закон, региональный уровень, законодательный орган       0.87      0.93      0.90       308
                закон, федеральный уровень, законодательный орган       0.95      0.97      0.96       280
 постановление, региональный уровень, глава исполнительной власти       0.74      0.82      0.78       305
       постановление, региональный уровень, законодательный орган       0.92      0.88      0.90       308
постановление, региональный уровень, органы исполнительной власти       0.99      0.96      0.97       237
               постановление, региональный уровень, правительство       0.72      0.77      0.75       311
                постановление, федеральный уровень, правительство       0.89      0.88      0.88       292
       приказ, региональный уровень, органы исполнительной власти       0.88      0.87      0.87       314
        приказ, федеральный уровень,

In [ ]:
fr_ppl_clf = Pipeline([

    ('rf_clf', RandomForestClassifier(random_state=42))
])

parameters = {
              'rf_clf__n_estimators':[300, 500, 700],
              'rf_clf__max_depth':[3, 5, 8, 11],


              }
model = GridSearchCV(fr_ppl_clf, parameters, cv=4, n_jobs=-1).fit(X_train[:2000], y_train[:2000])
print('Best score and parameter combination:')
print(model.best_score_, model.best_params_)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning:

`should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.



Best score and parameter combination:
0.7309999999999999 {'rf_clf__max_depth': 11, 'rf_clf__n_estimators': 500}


In [ ]:
fr_ppl_clf = Pipeline([

    ('sgd_clf', RandomForestClassifier(n_estimators=500,max_depth=11,  random_state=42))
])
fr_ppl_clf.fit(X_train, y_train)


predicted_fr_val = fr_ppl_clf.predict(X_valid)
print_metrics(y_valid, predicted_fr_val)

                                                                   precision    recall  f1-score   support

               закон, региональный уровень, законодательный орган       0.78      0.93      0.85       308
                закон, федеральный уровень, законодательный орган       0.92      0.96      0.94       280
 постановление, региональный уровень, глава исполнительной власти       0.78      0.70      0.74       305
       постановление, региональный уровень, законодательный орган       0.92      0.89      0.90       308
постановление, региональный уровень, органы исполнительной власти       0.97      0.90      0.93       237
               постановление, региональный уровень, правительство       0.63      0.77      0.69       311
                постановление, федеральный уровень, правительство       0.83      0.90      0.87       292
       приказ, региональный уровень, органы исполнительной власти       0.90      0.88      0.89       314
        приказ, федеральный уровень,